# Clustering

In den letzten Teilen hast du dich ausführlich mit Topic Models beschäftigt. Bei Textdaten ist das eine äußerst sinnvolle Möglichkeit, wie du Strukturen entdecken kannst.

Bei strukturierten Daten hingegen wird häufig mit Clustering gearbeitet. Auch Textdaten kannst du clustern.

## Daten einladen

Wie gewohnt lädst du die linguistisch analysierten Daten ein:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

Anschließend führst du die Vektorisierung durch. Das kann auch wieder einen Augenblick dauern:

In [ ]:
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=5)
tfidf_vectors = tfidf_vectorizer.fit_transform(df["nav"])
tfidf_vectors

Durch die Nutzung von `min_df=5` ist die Matrix einigermaßen übersichtlich geblieben!

## Clustering mit K-Means durchführen

Die Clusterin-Algorithmen bei `scikit-learn` nutzen das identische API wie die Topic Models. Das ist natürlich äußerst praktisch, weil du so gut wie nichts ändern musst.

Für größere Datenmengen kannst du `MiniBatchKMeans` verwenden, bei deutlich weniger als 10.000 Dokumenten solltest du besser `KMeans` selbst verwenden. Die `import`-Anweisung ist so gebaut, dass dur nur dort eine Ersetzung vornehmen musst:

In [ ]:
from sklearn.cluster import MiniBatchKMeans as KMeans
k_means = KMeans(n_clusters=10, random_state=42)
k_means.fit(tfidf_vectors)

Als Ergebnis steht dir nun eine `labels_`-Variable zur Verfügung, die dir für jedes Dokument sagt, zu welchem Cluster es gehört:

In [ ]:
k_means.labels_

Mithilfe von `numpy` kannst du die Multiplizäten ermitteln:

In [ ]:
import numpy as np
np.unique(k_means.labels_, return_counts=True)

Der Cluster `2` ist deutlich größer als die anderen. Das ist ein Effekt, den du häufiger bei Clustering beobachten kannst: ein Cluster fungiert als *Sammler*, der einfach alles aufnimmt. In diesem Fall ist es gar nicht so ausgeprägt, weli ein anderer Cluster (`0`) auch ziemlich groß ist.

Wenn du die TF/IDF-Gewichte der Cluster aufsummierst, findest du die dominierenden Wörter und kannst diese in Wordclouds einzeichnen:

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
def wordcloud_clusters(model, vectors, features, no_top_words=40):
    for cluster in np.unique(model.labels_):
        size = {}
        words = vectors[model.labels_ == cluster].sum(axis=0).A[0]
        largest = words.argsort()[::-1] # invert sort order
        for i in range(0, no_top_words):
            size[features[largest[i]]] = abs(words[largest[i]])
        wc = WordCloud(background_color="white", max_words=100, width=960, height=540)
        wc.generate_from_frequencies(size)
        plt.figure(figsize=(12,12))
        plt.imshow(wc, interpolation='bilinear')
        plt.axis("off")

In diesem Fall ist das Ergebnis richtig gut:

In [ ]:
wordcloud_clusters(k_means, tfidf_vectors, tfidf_vectorizer.get_feature_names())

## Clustering ist gut geeignet für kurze Texte

Da beim Clustering jedem Dokument genau ein Cluster zugeordnet wird, ist es etwas rigoroser als die Topic Models. Häufig ist es auch für Menschen schwer, einem Dokument nur ein einziges Thema zuzuordnen.

Bei kurzen Dokumenten oder solchen, die klar definierte Themen haben, geht das hingegen ziemlich gut. Daher kannst du auch für den Heise-Newsticker damit gute Ergebnisse erzielen, die denen des Topic Models sehr nahe kommen. Es gibt auch noch ein paar Vorteile: so kannst du *hierarchisch clustern*, weil du für jedes Dokument den Cluster kennst.